# **ROB 535 Perception Task 1**

**1) Importing Libraries**

In [ ]:
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


**2) Load Data CSV and Images**

In [ ]:
train = pd.read_csv("train_labels.csv")
validation = pd.read_csv("test_labels.csv")

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  6815 non-null   object
 1   width     6815 non-null   int64 
 2   height    6815 non-null   int64 
 3   class     6815 non-null   object
 4   xmin      6815 non-null   int64 
 5   ymin      6815 non-null   int64 
 6   xmax      6815 non-null   int64 
 7   ymax      6815 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 426.1+ KB


In [ ]:
# train_folder = "images/cropped_train"
# validation_folder = "images/cropped_test"

train_folder = "images/train"
validation_folder = "images/test"
test_folder = "images/predict_test"

In [ ]:
# Data generator to augment train data
train_gen = ImageDataGenerator(
rotation_range=45,
rescale=1./255,
horizontal_flip=True
)
validation_gen = ImageDataGenerator(rescale = 1.255)

test_gen = ImageDataGenerator(rescale = 1.255)

In [ ]:
train_data = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = "filename", 
y_col = "class", seed = 42,
shuffle = True, 
class_mode="categorical",
target_size = (300, 300))

validation_data = validation_gen.flow_from_dataframe(dataframe = validation, 
directory = validation_folder, x_col = "filename", 
y_col = "class",
shuffle = False, 
class_mode="categorical",
target_size = (300, 300))

test_data = test_gen.flow_from_directory(test_folder, target_size=(300, 300), class_mode=None, shuffle=False)

Found 6815 validated image filenames belonging to 3 classes.
Found 758 validated image filenames belonging to 3 classes.
Found 2631 images belonging to 23 classes.


**3) Load CNN Models**

In [ ]:
model_resnet = keras.applications.ResNet50V2(
    weights=None,  
    # input_shape=(150, 150, 3),
    input_shape=(300, 300, 3),
    include_top=False)  

model_vgg19 = keras.applications.VGG19(
    weights="imagenet",  
    input_tensor=None,
    input_shape=(300, 300, 3),
    include_top=False,
    classes=1000,
    classifier_activation="softmax")  

model_inceptionv3 = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(300, 300, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


**4) Train Models**

In [ ]:
model_resnet.trainable = True
model_vgg19.trainable = True
model_inceptionv3.trainable = True

In [ ]:
inputs = keras.Input(shape=(300, 300, 3))

In [ ]:
x1 = tf.keras.applications.resnet50.preprocess_input(inputs)
x2 = tf.keras.applications.vgg19.preprocess_input(inputs)
x3 = tf.keras.applications.inception_v3.preprocess_input(inputs)


In [ ]:
x1 = model_resnet(x1, training=True)
x1 = keras.layers.GlobalAveragePooling2D()(x1)
x1= keras.layers.Dropout(0.5)(x1)

x2 = model_vgg19(x2, training=True)
x2 = keras.layers.GlobalAveragePooling2D()(x2)
x2= keras.layers.Dropout(0.5)(x2)

x3 = model_inceptionv3(x3, training=True)
x3 = keras.layers.GlobalAveragePooling2D()(x3)
x3= keras.layers.Dropout(0.5)(x3)


outputs_resnet = keras.layers.Dense(3, activation=('softmax'))(x1)
model_resnet = keras.Model(inputs, outputs_resnet)
outputs_vgg19 = keras.layers.Dense(3, activation=('softmax'))(x2)
model_vgg19 = keras.Model(inputs, outputs_vgg19)
outputs_inceptionv3 = keras.layers.Dense(3, activation=('softmax'))(x3)
model_inceptionv3 = keras.Model(inputs, outputs_inceptionv3)

# x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dropout(0.2)(x)  
# outputs = keras.layers.Dense(3, activation=('softmax'))(x)
# model = keras.Model(inputs, outputs)

In [ ]:
model_resnet.summary()
model_vgg19.summary()
model_inceptionv3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 300, 300, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 300, 300, 3)      0         
                                                                 
 resnet50v2 (Functional)     (None, 10, 10, 2048)      23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0     

In [ ]:
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
batch_size= 100
epochs=15
learn_rate=.001
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


model_resnet.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model_vgg19.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model_inceptionv3.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='training-checkpoints/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.LearningRateScheduler(scheduler)
]

In [ ]:
model_resnet.fit(train_data, validation_data=validation_data, epochs=epochs, callbacks=my_callbacks)

In [ ]:
model_vgg19.fit(train_data, validation_data=validation_data, epochs=epochs, callbacks=my_callbacks)

In [ ]:
model_inceptionv3.fit(train_data, validation_data=validation_data, epochs=epochs, callbacks=my_callbacks)

**5) Making Predictions**

In [ ]:
predictions_resnet = model_resnet.predict(test_data)
model_resnet.save("model-resnet-noweights")

In [ ]:
predictions_vgg19 = model_vgg19.predict(test_data)
model_vgg19.save("model-vgg19")

In [ ]:
predictions_inceptionv3 = model_inceptionv3.predict(test_data)
model_inceptionv3.save("model-inceptionv3")

In [ ]:
import numpy as np

**6) Store Predictions in CSV**

In [ ]:
outputs_resnet = np.argmax(predictions_resnet,axis=1)
outputs_vgg19 = np.argmax(predictions_vgg19,axis=1)
outputs_inceptionv3 = np.argmax(predictions_inceptionv3,axis=1)

In [ ]:
some_model1 = keras.models.load_model("model-resnet-noweights")
some_model2 = keras.models.load_model("model-vgg19")
some_model3 = keras.models.load_model("model-inceptionv3")

In [ ]:
df = pd.DataFrame(outputs_resnet)
df.to_csv('prediction_outputs_resnet-noweights.csv')

df = pd.DataFrame(outputs_vgg19)
df.to_csv('prediction_outputs_vgg19.csv')

df = pd.DataFrame(outputs_inceptionv3)
df.to_csv('prediction_outputs_inceptionv3.csv')